In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# 한글 폰트 설정
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False

# 데이터 경로 설정
DATA_PATH = Path('../data')

print("📂 데이터 파일 목록:")
for file in DATA_PATH.glob('*.csv'):
    print(f"  - {file.name}")


📂 데이터 파일 목록:
  - 월별 품목별 국가별 수출입실적(2023년).csv
  - 월별_품목별_국가별 수출입실적(2021년).csv
  - 월별_품목별_국가별 수출입실적(2022년).csv
  - 한국무역보험공사_국가별 단기수출보험 보상현황_20240630.csv
  - 한국무역보험공사_국가별 업종별 위험지수(RISK INDEX)_20250501.csv


In [2]:
# 수출입실적 데이터 로딩 (인코딩 문제 해결)
def load_csv_with_encoding(file_path, file_name):
    """다양한 인코딩으로 CSV 파일 로딩 시도"""
    encodings = ['cp949', 'euc-kr', 'utf-8', 'utf-8-sig', 'latin1']
    
    for encoding in encodings:
        try:
            df = pd.read_csv(file_path, encoding=encoding)
            print(f"✅ {file_name} 로딩 성공 (인코딩: {encoding})")
            return df
        except UnicodeDecodeError:
            continue
        except Exception as e:
            print(f"❌ {file_name} 로딩 실패 ({encoding}): {e}")
            continue
    
    print(f"❌ {file_name} 모든 인코딩 시도 실패")
    return None

# 수출입실적 파일들 로딩
df_export_2021 = load_csv_with_encoding(DATA_PATH / '월별_품목별_국가별 수출입실적(2021년).csv', '2021년 수출입실적')
df_export_2022 = load_csv_with_encoding(DATA_PATH / '월별_품목별_국가별 수출입실적(2022년).csv', '2022년 수출입실적')
df_export_2023 = load_csv_with_encoding(DATA_PATH / '월별 품목별 국가별 수출입실적(2023년).csv', '2023년 수출입실적')

# 데이터 로딩 성공 여부 확인
loaded_dfs = []
if df_export_2021 is not None:
    df_export_2021['연도'] = 2021
    loaded_dfs.append(df_export_2021)
    print(f"2021년: {df_export_2021.shape}")

if df_export_2022 is not None:
    df_export_2022['연도'] = 2022
    loaded_dfs.append(df_export_2022)
    print(f"2022년: {df_export_2022.shape}")

if df_export_2023 is not None:
    df_export_2023['연도'] = 2023
    loaded_dfs.append(df_export_2023)
    print(f"2023년: {df_export_2023.shape}")

# 로딩된 데이터들 통합
if loaded_dfs:
    df_export_all = pd.concat(loaded_dfs, ignore_index=True)
    print(f"\n📈 통합 수출입 데이터: {df_export_all.shape}")
    
    # 첫 번째 로딩된 데이터의 구조 확인
    print("\n📊 데이터 샘플:")
    print(loaded_dfs[0].head())
    print(f"\n컬럼: {loaded_dfs[0].columns.tolist()}")
else:
    print("❌ 수출입 데이터를 하나도 로딩할 수 없습니다!")


✅ 2021년 수출입실적 로딩 성공 (인코딩: cp949)
✅ 2022년 수출입실적 로딩 성공 (인코딩: utf-8)
✅ 2023년 수출입실적 로딩 성공 (인코딩: cp949)
2021년: (511253, 8)
2022년: (504923, 8)
2023년: (500978, 8)

📈 통합 수출입 데이터: (1517154, 8)

📊 데이터 샘플:
        월별  세번4단위    국가  수출중량  수입중량  수출액  수입액    연도
0  2021-03    101  네덜란드     0     2    0  130  2021
1  2021-05    101  네덜란드     0     2    0  141  2021
2  2021-06    101  네덜란드     0     1    0  162  2021
3  2021-07    101  네덜란드     0     2    0  150  2021
4  2021-08    101  네덜란드     0     1    0   70  2021

컬럼: ['월별', '세번4단위', '국가', '수출중량', '수입중량', '수출액', '수입액', '연도']


In [3]:
# 보상현황 데이터 로딩 (인코딩 문제 해결)
df_claims_2024 = load_csv_with_encoding(
    DATA_PATH / '한국무역보험공사_국가별 단기수출보험 보상현황_20240630.csv', 
    '보상현황 데이터'
)

if df_claims_2024 is not None:
    print(f"Shape: {df_claims_2024.shape}")
    
    # 데이터 구조 확인
    print("\n📊 보상현황 데이터 샘플:")
    print(df_claims_2024.head())
    print(f"\n컬럼: {df_claims_2024.columns.tolist()}")
    
    # 보상현황 데이터 전처리
    df_claims_2024.columns = ['년분기', '국가명', '보상금', '회수금']
    print(f"\n🔧 컬럼명 정리 완료: {df_claims_2024.columns.tolist()}")
    
    # 연도분기 분리 (NaN 값 처리)
    # 연도 추출
    df_claims_2024['연도'] = df_claims_2024['년분기'].str.extract('(\d{4})')
    df_claims_2024['연도'] = pd.to_numeric(df_claims_2024['연도'], errors='coerce').fillna(2024).astype(int)
    
    # 분기 추출 (여러 패턴 시도)
    # 패턴 1: "X분기" 형태
    분기1 = df_claims_2024['년분기'].str.extract('(\d)분기')
    # 패턴 2: "상반기", "하반기" 형태
    분기2 = df_claims_2024['년분기'].str.replace('상반기', '1').str.replace('하반기', '2')
    분기2 = pd.to_numeric(분기2.str.extract('(\d)')[0], errors='coerce')
    
    # 두 패턴 결합
    df_claims_2024['분기'] = pd.to_numeric(분기1[0], errors='coerce').fillna(분기2).fillna(1).astype(int)
    
    print(f"📅 연도분기 분리 완료:")
    print(f"- 연도 범위: {df_claims_2024['연도'].min()}-{df_claims_2024['연도'].max()}")
    print(f"- 분기 분포: {df_claims_2024['분기'].value_counts().sort_index().to_dict()}")
    
    # 숫자형 변환 (콤마, 공백, 기타 문자 제거)
    numeric_cols = ['보상금', '회수금']
    for col in numeric_cols:
        if df_claims_2024[col].dtype == 'object':
            # 문자열로 변환 후 숫자가 아닌 문자 제거
            df_claims_2024[col] = df_claims_2024[col].astype(str).str.replace(',', '').str.replace(' ', '').str.replace('원', '')
            # 숫자만 추출
            df_claims_2024[col] = df_claims_2024[col].str.extract('(\d+)')[0]
            df_claims_2024[col] = pd.to_numeric(df_claims_2024[col], errors='coerce').fillna(0)
    
    # 0으로 나누기 방지한 비율 계산
    total_amount = df_claims_2024['보상금'] + df_claims_2024['회수금']
    df_claims_2024['보상률'] = np.where(total_amount > 0, 
                                     df_claims_2024['보상금'] / total_amount, 0)
    
    total_claims = df_claims_2024['보상금'].sum()
    df_claims_2024['손실률'] = np.where(total_claims > 0,
                                     df_claims_2024['보상금'] / total_claims, 0)
    
    print(f"💰 숫자형 변환 완료:")
    print(f"- 총 보상금: {df_claims_2024['보상금'].sum():,.0f}원")
    print(f"- 총 회수금: {df_claims_2024['회수금'].sum():,.0f}원")
    print(f"- 평균 보상률: {df_claims_2024['보상률'].mean():.4f}")
    
    print("\n📈 주요 통계:")
    print(df_claims_2024[['보상금', '회수금', '보상률']].describe())
else:
    print("❌ 보상현황 데이터를 로딩할 수 없습니다!")


✅ 보상현황 데이터 로딩 성공 (인코딩: cp949)
Shape: (1235, 4)

📊 보상현황 데이터 샘플:
          구분   국가명         보험금         회수금
0  2021년 상반기  대한민국           0           0
1  2021년 상반기    대만     3055140    36574531
2  2021년 상반기    중국  1461543917  1791600907
3  2021년 상반기    홍콩   293886636     5853681
4  2021년 상반기    일본    54277920    47212682

컬럼: ['구분', '국가명', '보험금', '회수금']

🔧 컬럼명 정리 완료: ['년분기', '국가명', '보상금', '회수금']
📅 연도분기 분리 완료:
- 연도 범위: 2021-2024
- 분기 분포: {2: 1235}
💰 숫자형 변환 완료:
- 총 보상금: 272,167,353,323원
- 총 회수금: 170,206,556,495원
- 평균 보상률: 0.2190

📈 주요 통계:
                보상금           회수금          보상률
count  1.235000e+03  1.235000e+03  1235.000000
mean   2.203784e+08  1.378191e+08     0.218960
std    1.035501e+09  1.188133e+09     0.374047
min    0.000000e+00 -2.194602e+07     0.000000
25%    0.000000e+00  0.000000e+00     0.000000
50%    0.000000e+00  0.000000e+00     0.000000
75%    3.270208e+07  1.271672e+07     0.366200
max    1.772370e+10  3.716997e+10     1.000000


In [4]:
# AI 위험지수 데이터 로딩 (인코딩 문제 해결)
df_risk_2025 = load_csv_with_encoding(
    DATA_PATH / '한국무역보험공사_국가별 업종별 위험지수(RISK INDEX)_20250501.csv', 
    'AI 위험지수 데이터'
)

if df_risk_2025 is not None:
    print(f"Shape: {df_risk_2025.shape}")
    
    # 데이터 구조 확인
    print("\n📊 AI 위험지수 데이터 샘플:")
    print(df_risk_2025.head())
    print(f"\n컬럼: {df_risk_2025.columns.tolist()}")
    
    # AI 위험지수 데이터 전처리
    df_risk_2025.columns = ['기준년월', '국가명', '업종명', '위험지수']
    print(f"\n🔧 컬럼명 정리 완료: {df_risk_2025.columns.tolist()}")
    
    # 위험지수가 숫자가 아닌 경우 변환
    if df_risk_2025['위험지수'].dtype == 'object':
        df_risk_2025['위험지수'] = pd.to_numeric(df_risk_2025['위험지수'], errors='coerce')
    
    # 위험지수 분포 확인
    print("\n📊 위험지수 분포:")
    print(df_risk_2025['위험지수'].value_counts().sort_index())
    
    # 국가별 평균 위험지수
    country_risk = df_risk_2025.groupby('국가명')['위험지수'].agg(['mean', 'std', 'count']).round(2)
    print(f"\n🌍 국가별 평균 위험지수 (상위 10개국):")
    print(country_risk.sort_values('mean', ascending=False).head(10))
else:
    print("❌ AI 위험지수 데이터를 로딩할 수 없습니다!")


✅ AI 위험지수 데이터 로딩 성공 (인코딩: cp949)
Shape: (24584, 4)

📊 AI 위험지수 데이터 샘플:
      기준년월 국가한글명                    업종한글명  위험지수(Risk Index)
0  2025-04    가나          고무 및 플라스틱제품 제조업                 5
1  2025-04    가나                    금속 광업                 5
2  2025-04    가나              도매 및 상품 중개업                 4
3  2025-04    가나           자동차 및 트레일러 제조업                 4
4  2025-04    가나  화학물질 및 화학제품 제조업; 의약품 제외                 5

컬럼: ['기준년월', '국가한글명', '업종한글명', '위험지수(Risk Index)']

🔧 컬럼명 정리 완료: ['기준년월', '국가명', '업종명', '위험지수']

📊 위험지수 분포:
위험지수
1    2943
2    6727
3    5126
4    7352
5    2436
Name: count, dtype: int64

🌍 국가별 평균 위험지수 (상위 10개국):
         mean  std  count
국가명                      
카메룬       5.0  0.0      8
시에라리온     5.0  0.0      8
타지키스탄     5.0  0.0     16
부룬디       5.0  0.0      8
부르키나파소    5.0  0.0     32
리비아       5.0  0.0      8
마다가스카르    5.0  0.0      4
라이베리아     5.0  0.0     13
키르기즈공화국   5.0  0.0     16
말리        5.0  0.0     24


In [5]:
# 4-1. 수출 데이터 월별 집계
if 'df_export_all' in locals() and df_export_all is not None:
    # 수출액이 숫자형이 아닌 경우 변환
    numeric_cols = ['수출액', '수출중량', '수입액', '수입중량']
    for col in numeric_cols:
        if col in df_export_all.columns and df_export_all[col].dtype == 'object':
            df_export_all[col] = df_export_all[col].astype(str).str.replace(',', '').str.replace(' ', '')
            df_export_all[col] = pd.to_numeric(df_export_all[col], errors='coerce').fillna(0)
    
    # 국가별 월별 수출액 집계
    export_monthly = df_export_all.groupby(['연도', '월별', '국가']).agg({
        '수출액': 'sum',
        '수출중량': 'sum'
    }).reset_index()
    
    # 월별을 datetime으로 변환 (더 안전한 방법)
    try:
        # 월별 데이터 형태 확인
        print(f"📅 월별 데이터 샘플: {export_monthly['월별'].head().tolist()}")
        
        # 다양한 형식 처리
        if export_monthly['월별'].dtype == 'object':
            # 'YYYY-MM' 형태인 경우
            if export_monthly['월별'].str.contains('-').any():
                month_part = export_monthly['월별'].str.split('-').str[1]
                export_monthly['년월'] = pd.to_datetime(export_monthly['연도'].astype(str) + '-' + 
                                                      month_part.astype(str) + '-01', errors='coerce')
            else:
                # 숫자 형태인 경우 (1, 2, 3, ... 12)
                month_part = export_monthly['월별'].astype(str).str.zfill(2)  # 01, 02 형태로 변환
                export_monthly['년월'] = pd.to_datetime(export_monthly['연도'].astype(str) + '-' + 
                                                      month_part + '-01', errors='coerce')
        else:
            # 이미 숫자인 경우
            month_part = export_monthly['월별'].astype(str).str.zfill(2)
            export_monthly['년월'] = pd.to_datetime(export_monthly['연도'].astype(str) + '-' + 
                                                  month_part + '-01', errors='coerce')
        
        # NaT 값이 있는지 확인
        nat_count = export_monthly['년월'].isna().sum()
        if nat_count > 0:
            print(f"⚠️ 날짜 변환 실패 건수: {nat_count}")
            # 실패한 경우 문자열로 대체
            failed_mask = export_monthly['년월'].isna()
            export_monthly.loc[failed_mask, '년월'] = (export_monthly.loc[failed_mask, '연도'].astype(str) + '-' + 
                                                     export_monthly.loc[failed_mask, '월별'].astype(str))
        
        print(f"✅ 날짜 변환 완료: {export_monthly['년월'].head().tolist()}")
        
    except Exception as e:
        print(f"⚠️ 날짜 변환 오류: {e}")
        # 최종 대안: 문자열 조합
        export_monthly['년월'] = export_monthly['연도'].astype(str) + '-' + export_monthly['월별'].astype(str)
    
    print(f"✅ 수출 데이터 집계 완료: {export_monthly.shape}")
    print("📊 집계된 수출 데이터 샘플:")
    print(export_monthly.head())
else:
    print("❌ 수출 데이터가 없어 집계를 수행할 수 없습니다!")

    print("📅 수출 월별 집계 완료")
    print(f"Shape: {export_monthly.shape}")
    print(export_monthly.head())
    
    # 수출 증가율 계산 (전년동월 대비) - 안전하게 처리
    try:
        print("\n📈 수출증가율 계산 시작...")
        export_monthly = export_monthly.sort_values(['국가', '년월'])
        export_monthly['전년동월_수출액'] = export_monthly.groupby('국가')['수출액'].shift(12)
        
        # 0으로 나누기 방지
        mask = export_monthly['전년동월_수출액'] > 0
        export_monthly['수출증가율'] = 0.0  # 기본값
        export_monthly.loc[mask, '수출증가율'] = (
            (export_monthly.loc[mask, '수출액'] - export_monthly.loc[mask, '전년동월_수출액']) / 
            export_monthly.loc[mask, '전년동월_수출액'] * 100
        ).round(2)
        
        print("✅ 수출증가율 계산 완료")
        valid_growth = export_monthly[export_monthly['수출증가율'] != 0].head(10)
        if not valid_growth.empty:
            print("📊 수출증가율 샘플:")
            print(valid_growth[['국가', '년월', '수출액', '수출증가율']])
        else:
            print("⚠️ 계산된 수출증가율이 없습니다 (데이터 기간 부족 가능)")
            
    except Exception as e:
        print(f"⚠️ 수출증가율 계산 오류: {e}")
        export_monthly['수출증가율'] = 0.0  # 기본값으로 설정


📅 월별 데이터 샘플: ['2021-01', '2021-01', '2021-01', '2021-01', '2021-01']
✅ 날짜 변환 완료: [Timestamp('2021-01-01 00:00:00'), Timestamp('2021-01-01 00:00:00'), Timestamp('2021-01-01 00:00:00'), Timestamp('2021-01-01 00:00:00'), Timestamp('2021-01-01 00:00:00')]
✅ 수출 데이터 집계 완료: (8316, 6)
📊 집계된 수출 데이터 샘플:
     연도       월별    국가    수출액   수출중량         년월
0  2021  2021-01    가나  21423  10066 2021-01-01
1  2021  2021-01    가봉    944    153 2021-01-01
2  2021  2021-01  가이아나   2072    485 2021-01-01
3  2021  2021-01   감비아    351     70 2021-01-01
4  2021  2021-01    건지      0      0 2021-01-01


In [6]:
# 4-2. 국가명 표준화 (3개 데이터셋 간 매칭)
print("🔧 국가명 표준화 작업")

# 데이터가 로딩되었는지 확인
datasets_loaded = []
if 'export_monthly' in locals() and export_monthly is not None:
    datasets_loaded.append(('export', export_monthly, '국가'))
if 'df_claims_2024' in locals() and df_claims_2024 is not None:
    datasets_loaded.append(('claims', df_claims_2024, '국가명'))
if 'df_risk_2025' in locals() and df_risk_2025 is not None:
    datasets_loaded.append(('risk', df_risk_2025, '국가명'))

print(f"✅ 로딩된 데이터셋: {len(datasets_loaded)}개")

if len(datasets_loaded) >= 2:
    # 각 데이터셋의 고유 국가명 확인
    country_sets = {}
    for name, df, col in datasets_loaded:
        countries = set(df[col].unique())
        country_sets[name] = countries
        print(f"{name} 데이터 국가 수: {len(countries)}")
    
    # 공통 국가 찾기 (최소 2개 이상의 데이터셋에서 공통인 국가)
    if len(datasets_loaded) >= 3:
        common_countries = set.intersection(*country_sets.values())
    else:
        # 2개 데이터셋만 있는 경우
        common_countries = set.intersection(*list(country_sets.values()))
    
    print(f"\n🌍 공통 국가 수: {len(common_countries)}")
    if common_countries:
        print(f"공통 국가 (상위 10개): {list(common_countries)[:10]}")
        
        # 분석용 데이터셋 필터링 (공통 국가만)
        filtered_datasets = {}
        for name, df, col in datasets_loaded:
            filtered = df[df[col].isin(common_countries)].copy()
            filtered_datasets[name] = filtered
            print(f"✅ {name} 데이터 필터링: {filtered.shape}")
    else:
        print("❌ 공통 국가가 없습니다!")
        # 각 데이터셋을 그대로 저장
        filtered_datasets = {name: df for name, df, col in datasets_loaded}
else:
    print("❌ 로딩된 데이터셋이 부족합니다!")


🔧 국가명 표준화 작업
✅ 로딩된 데이터셋: 3개
export 데이터 국가 수: 252
claims 데이터 국가 수: 196
risk 데이터 국가 수: 165

🌍 공통 국가 수: 129
공통 국가 (상위 10개): ['파나마', '멕시코', '미얀마', '대만', '싱가포르', '에스토니아', '이스라엘', '바하마', '코스타리카', '파키스탄']
✅ export 데이터 필터링: (4642, 6)
✅ claims 데이터 필터링: (887, 8)
✅ risk 데이터 필터링: (22333, 4)


In [7]:
# 4-3. 분석용 최종 데이터셋 저장
print("💾 분석용 데이터셋 저장")

# Output 폴더 생성 (존재하지 않을 경우)
OUTPUT_PATH = Path('../output')
OUTPUT_PATH.mkdir(exist_ok=True)

if 'filtered_datasets' in locals() and filtered_datasets:
    saved_files = []
    
    # 각 데이터셋을 저장
    if 'export' in filtered_datasets:
        filtered_datasets['export'].to_csv(OUTPUT_PATH / 'export_data_processed.csv', index=False, encoding='cp949')
        saved_files.append("- export_data_processed.csv: 수출입 실적 (2021-2023)")
    
    if 'claims' in filtered_datasets:
        filtered_datasets['claims'].to_csv(OUTPUT_PATH / 'claims_data_processed.csv', index=False, encoding='cp949')
        saved_files.append("- claims_data_processed.csv: 보상현황 (2024)")
    
    if 'risk' in filtered_datasets:
        filtered_datasets['risk'].to_csv(OUTPUT_PATH / 'risk_data_processed.csv', index=False, encoding='cp949')
        saved_files.append("- risk_data_processed.csv: AI 위험지수 (2025)")
    
    # 공통 국가 리스트 저장 (있는 경우)
    if 'common_countries' in locals() and common_countries:
        pd.DataFrame(list(common_countries), columns=['국가명']).to_csv(
            OUTPUT_PATH / 'common_countries.csv', index=False, encoding='cp949')
        saved_files.append("- common_countries.csv: 공통 국가 리스트")
    
    print("✅ 전처리 완료! 저장된 파일:")
    for file in saved_files:
        print(file)
    
    # 최종 데이터 요약
    print(f"\n📊 최종 데이터 요약:")
    if 'common_countries' in locals() and common_countries:
        print(f"- 분석 대상 국가: {len(common_countries)}개국")
    
    if 'export' in filtered_datasets:
        export_data = filtered_datasets['export']
        if '년월' in export_data.columns:
            print(f"- 수출 데이터 기간: 2021-2023 ({export_data['년월'].nunique()}개월)")
        else:
            print(f"- 수출 데이터: {export_data.shape}")
    
    if 'claims' in filtered_datasets:
        claims_data = filtered_datasets['claims']
        if '연도' in claims_data.columns:
            print(f"- 보상 데이터 기간: {claims_data['연도'].min()}-{claims_data['연도'].max()}")
        else:
            print(f"- 보상 데이터: {claims_data.shape}")
    
    if 'risk' in filtered_datasets:
        print(f"- AI 위험지수: 2025년 4월 기준")
        
else:
    print("❌ 저장할 데이터가 없습니다!")

print("\n🎯 다음 단계: 02_lag_effect_analysis.ipynb 실행!")


💾 분석용 데이터셋 저장
✅ 전처리 완료! 저장된 파일:
- export_data_processed.csv: 수출입 실적 (2021-2023)
- claims_data_processed.csv: 보상현황 (2024)
- risk_data_processed.csv: AI 위험지수 (2025)
- common_countries.csv: 공통 국가 리스트

📊 최종 데이터 요약:
- 분석 대상 국가: 129개국
- 수출 데이터 기간: 2021-2023 (36개월)
- 보상 데이터 기간: 2021-2024
- AI 위험지수: 2025년 4월 기준

🎯 다음 단계: 02_lag_effect_analysis.ipynb 실행!
